In [65]:
import tensorflow as tf
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pathlib
import librosa
import matplotlib
import matplotlib as plt
from IPython.display import Audio
from tensorflow.keras.models import load_model
import pathlib
print(tf.__version__)

2.10.0


In [66]:
%rm -rf ../content/testset

In [67]:
test_path='testset/'

genre_list=['bhojpuri_pop',
 'carnatic',
 'classic_bollywood',
 'desi_pop',
 'ghazal',
 'hindustani_classical',
 'indian_indie',
 'punjabi_hip_hop',
 'sufi',
 'tamil_pop']

In [68]:
file_name = "../content/spectrograms6secnew/bhojpuri_pop/Ankush Raja - Dar kekra se ba14.png"

filenameparts=file_name.split('/')
song_name=filenameparts[-1]
song_name

'Ankush Raja - Dar kekra se ba14.png'

In [69]:
%rm -rf testset

In [70]:
def save_spectrogram(block,sr,genre,song_name,counter):
  matplotlib.use('Agg')
  S = librosa.feature.melspectrogram(y=block, sr=sr, n_mels=128,fmax=8000)
  plt.figure(figsize=(6, 3))
  librosa.display.specshow(librosa.power_to_db(S,ref=np.max),y_axis='mel', fmax=8000,x_axis='time')
  # plt.plot()
  song_name=song_name.replace('.mp3','')
  file_name=f'testset/{genre}/'+song_name+str(counter)+'.png'
  plt.savefig(file_name)
  plt.close()

In [71]:
audio, sr = librosa.load(file_name)

/home/ayushmaankaushik/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


NoBackendError: 

In [72]:
Audio(data=audio, rate=sr)

NameError: name 'audio' is not defined

In [73]:
import loadsong

loadsong.make_spectrograms(audio,sr,song_name)

ModuleNotFoundError: No module named 'loadsong'

In [74]:
os.mkdir('testset')
dest_path='testset'

for genre in genre_list:
  os.mkdir(f'testset/{genre}')

In [75]:
data_dir_test="../content/spectrograms6secnew/"

In [76]:
dest_path='../content/spectrograms6secnew'

data_dir=pathlib.Path(dest_path)
data_dir

PosixPath('../content/spectrograms6secnew')

In [80]:
#defining dataset dimensions
batch_size=64
image_height=100
image_width=200

#loading datasets

test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size,
  shuffle="False"
  )

test_ds_cnn = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  seed=123,
  image_size=(75, 150),
  batch_size=batch_size,
  shuffle="False"
  )

genre_list=test_ds.class_names

Found 29990 files belonging to 10 classes.


2022-11-17 19:30:35.752392: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-17 19:30:35.752425: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-17 19:30:35.752443: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-11-17 19:30:35.753434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 29990 files belonging to 10 classes.


In [81]:
import tensorflow
load_model=tensorflow.keras.models.load_model

In [82]:
def ensemble_models(model_genres:list,model_input,model_input_cnn):
    #loading all models
    models=[]
    models.append(load_model('../FinalModel/models/model1/'))
    models.append(load_model('../FinalModel/models/model2/'))
    models.append(load_model('../FinalModel/models/model3/'))
    models.append(load_model('../FinalModel/models/model4/'))

    #prediction
    model1=models[0].predict(model_input)
    model2=models[1].predict(model_input)
    model3=models[2].predict(model_input)
    model4=models[3].predict(model_input_cnn)

    avgEnsemble=(model1+model2+model3+model4)/len(models)

    avgLis=[]
    for i in avgEnsemble:
        avgLis.append(i.argmax())
    
    most_confi=max(avgLis,key=avgLis.count)

    return model_genres[most_confi]

In [85]:
best_genre = ensemble_models(genre_list,test_ds,test_ds_cnn)
print(best_genre)

../content/spectrograms6secnew/bhojpuri_pop/Ankush Raja - Dar kekra se ba14.png Ankush Raja - Dar kekra se ba14.png
118/469 [======>.......................] - ETA: 1:12

In [84]:
from PIL import Image
from scipy import spatial

test_image1 = Image.open("../content/spectrograms6secnew/ghazal/Abida Parveen - Niyaten-e-Shauq Bhar Na20.png")
test_image2 = Image.open("../content/spectrograms6secnew/desi_pop/A.R. Rahman, Arijit Singh - Enna Sona18.png")

img_1 = test_image1.resize((100,200))
img_2 = test_image2.resize((100,200))

arr_1 = np.array(img_1)
arr_2 = np.array(img_2)

arr_1 = arr_1.flatten()
arr_2 = arr_2.flatten()

arr_1 = arr_1/255
arr_2 = arr_2/255

similarity = -1 * (spatial.distance.cosine(arr_1,arr_2) - 1)
print(similarity)

0.9856211725461667


In [20]:
def get_image_array(image):
    img = image.resize((100,200))
    img_arr = np.array(img)
    img_arr = img_arr.flatten()
    img_arr = img_arr/255
    return img_arr

In [54]:
def get_song_name(img_name):
    return ''.join((x for x in img_name[:-4] if not x.isdigit()))

compared_image = get_image_array(test_image1)
similarity_images = []
path = os.path.join(dest_path,best_genre)
prev_song_name = ''
for images in os.listdir(path):
    image_obj = Image.open(os.path.join(path,images))
    
    image2 = get_image_array(image_obj)
    similarity = -1 * (spatial.distance.cosine(compared_image,image2) - 1)
    image_name = images.split('\\')[-1]
    song_name = get_song_name(image_name)
    
    prev_song_name = song_name
    similarity_images.append([song_name,similarity])

In [62]:
df = pd.DataFrame(similarity_images,columns=["Name","Score"])
avgeraged_similarities = df.groupby('Name').mean('Score').reset_index()
suggestions = avgeraged_similarities.sort_values('Score',ascending=False).head(10)
# suggestions = df.sort_values('Score',ascending=False).head(10)
suggestions

,Name,Score
0,Abida Parveen - Niyaten-e-Shauq Bhar Na,0.987568
94,Talat Aziz - Ab Kya Ghazal Sunaoon,0.987326
44,Jagjit Singh - Koi Yeh Kaise Bataye,0.987156
88,Ravi Bal - Ishqe Noo Ched Na Bethi,0.987091
75,Nafad Sinan - Dastoor,0.987018
53,"Joell, Chinmayi Tripathi, Shriram Sampath - Da...",0.986811
46,Jagjit Singh - Shayad Main Zindagi Ki Sahar,0.986805
11,Bayaan - Farda,0.986783
32,Ghulam Ali - Roya,0.986781
64,Mehdi Hassan - Duniya Kisi Ke,0.986681
